In [2]:
import numpy as np
from numba import njit
from functools import partial

import pandas as pd
import vectorbt as vbt
import vectorbt as vbt
from vectorbt.utils.colors import adjust_opacity
from vectorbt.utils.enum_ import map_enum_fields
from vectorbt.base.reshape_fns import broadcast, flex_select_auto_nb, to_2d_array
from vectorbt.portfolio.enums import SizeType, Direction, NoOrder, OrderStatus, OrderSide
from vectorbt.portfolio import nb

import talib 

import sys
import os
function_essential_path = r"H:\Desktop\Environement_Trading_Developement\FunctionEssential"
function_s_score_path = r'H:\Desktop\Environement_Trading_Developement\Indicateur'
sys.path.append(function_essential_path)
sys.path.append(function_s_score_path)
import utils as us
import s_score as score
import market_regime as mr

In [3]:
##FONCTION##

In [30]:
def entries_trade(close, sscore, market_regime, seuil_sscore ):
    entries_long = [False] * len(close)
    entries_short = [False] * len(close)
    count = 10
    for i in range(len(close)):
        #if market_regime[i] >= 75 and count >= 10:

            if sscore[i] > seuil_sscore:
                entries_short[i] = True
                count = 0
            if sscore[i] < -1 * seuil_sscore:
                entries_long[i] = True
                count = 0
        #count = count +1

    return entries_long, entries_short


In [5]:
##INDICATEUR FACTORY##

In [39]:
##CLASS STRATEGIE##

@njit
def pre_sim_func_nb(c):
    entry_price = np.full(c.target_shape[1], np.nan, dtype=np.float64)  
    return (entry_price,)

@njit
def order_func_nb(c, entry_price, long, short, stop_loss, take_profit, size):
    price_now = nb.get_elem_nb(c, c.close)
    entry_long_now = nb.get_elem_nb(c, long)
    entry_short_now = nb.get_elem_nb(c, short)

    if entry_long_now:
        if c.position_now == 0:
            return nb.order_nb(
                size[c.col], 
                price=price_now,
                direction=Direction.LongOnly,
                fees=0.0005
            )
        elif c.position_now < 0:
            return nb.order_nb(
                -size[c.col],  
                price=price_now,
                direction=Direction.ShortOnly,
                fees=0.0005
            )  

    if entry_short_now:
        if c.position_now == 0:
            return nb.order_nb(
                size[c.col], 
                price=price_now,
                direction=Direction.ShortOnly,
                fees=0.0005
            )
        elif c.position_now > 0:
            return nb.order_nb(
                -size[c.col],  
                price=price_now,
                direction=Direction.LongOnly,
                fees=0.0005
            )

    if c.position_now > 0:
        SL = entry_price[c.col] * (1 - stop_loss[c.col])
        TP = entry_price[c.col] * (1 + take_profit[c.col])
        
        if price_now >= TP:
            return nb.order_nb(
                -size[c.col],
                price=TP,
                direction=Direction.LongOnly,
                fees=0.0005
            )
        elif price_now <= SL:
            return nb.order_nb(
                -size[c.col],
                price=SL,
                direction=Direction.LongOnly,
                fees=0.0005
            )

    if c.position_now < 0:
        SL = entry_price[c.col] * (1 + stop_loss[c.col])
        TP = entry_price[c.col] * (1 - take_profit[c.col])
        
        if price_now >= SL:
            return nb.order_nb(
                -size[c.col],
                price=SL,
                direction=Direction.ShortOnly,
                fees=0.0005
            )
        elif price_now <= TP:
            return nb.order_nb(
                -size[c.col],
                price=TP,
                direction=Direction.ShortOnly,
                fees=0.0005
            )

    return NoOrder

@njit
def post_order_func_nb(c, entry_price):
    if c.order_result.status == OrderStatus.Filled:
        entry_price[c.col] = c.order_result.price

class Strategie():

    def __init__(self, data, tickers, frequence_exec, frequence, multitimeframe):
        self.data = data
        self.tickers = tickers
        self.frequence_exec = frequence_exec
        self.frequences = frequence
        self.multitimeframe = multitimeframe  

    def backtest(self, sscore):
        entries_long_df = pd.DataFrame()
        entries_short_df = pd.DataFrame()
        exits_long_df = pd.DataFrame()
        exits_short_df = pd.DataFrame()
        close_df = pd.DataFrame()
        datetime_df = pd.DataFrame()

        for ticker in self.tickers:
            data = self.data[ticker]
            s_score_and_market_regime = sscore[ticker]

            close = data['close']
            s_score_ajuster = s_score_and_market_regime["adjusted_s_scores_vol"]
            marget_regime = s_score_and_market_regime["market_regime"]

            data_reset = data.reset_index()
            datetime = data_reset['Datetime']

            long, short = entries_trade(close, s_score_ajuster,marget_regime,3 )
            long = pd.Series(long, index=data['close'].index)
            short = pd.Series(short, index=data['close'].index)

            # us.print_trades(0,long,short,None,None,data)

            long = long.reset_index(drop=True)
            short = short.reset_index(drop=True)
            close = data['close'].reset_index(drop=True)
            datetime = datetime.reset_index(drop=True)
            entries_long_df[ticker] = long
            entries_short_df[ticker] = short
            close_df[ticker] = close
            datetime_df[ticker] = datetime

        close_df = close_df.replace(0, None).fillna(method='ffill')

        capital = 100000  
        risk_per_trade = 0.01  
        position_value = capital * risk_per_trade  

        size = np.array([position_value / close_df.iloc[-1, i] for i in range(len(self.tickers))])  

        stop_loss = np.array([0.02] * len(self.tickers))  
        take_profit = np.array([0.04] * len(self.tickers))  

        portfolio = vbt.Portfolio.from_order_func(
            close_df,  
            order_func_nb,  
            entries_long_df.values,  
            entries_short_df.values,  
            stop_loss,  
            take_profit,  
            size,  
            freq=self.frequence_exec,  
            init_cash=100000,  # Ajout du capital initial
            pre_sim_func_nb=pre_sim_func_nb,  
            post_order_func_nb=post_order_func_nb,  
            broadcast_named_args=dict(  
                long=entries_long_df.values,  
                short=entries_short_df.values,  
                stop_loss=stop_loss,  
                take_profit=take_profit,  
                size=size  
            )
        )


        us.rapport_backtest(portfolio,close_df,datetime_df,self.tickers)

        return portfolio


In [25]:
##UTILISATION DE LA STRAT##

tickers = ['GPDUSD']
DATA = us.get_data_forex(tickers,'H4',"2000-01-01","2011-01-01")

s_score_value_and_market_regime = {}
for ticker in tickers:
    data_traitement = DATA[ticker]
    nb_nan = data_traitement['close'].isna().sum()
    close = data_traitement['close'].dropna()
    close_without_index = close.reset_index()
    result = score.mean_reversion(close, close_without_index['Datetime'], 85, "VOL", 0.5, "EGARCH")
    market = mr.adaptive_market_regime(close)
    result_df = pd.DataFrame({
        's_score_value': result[0],
        'adjusted_s_scores_vol': result[1],
        'theta_vals': result[2],
        'mu_vals': result[3],
        'sigma_vals': result[4],
        'market_regime': market
    })
    nan_rows = pd.DataFrame([[float('nan')] * result_df.shape[1]] * nb_nan, columns=result_df.columns)
    result_df = pd.concat([result_df, nan_rows], ignore_index=True)

    s_score_value_and_market_regime[ticker] = result_df

Estimation des paramètres OU:   0%|          | 0/13188 [00:00<?, ?it/s]

  0%|          | 0/13188 [00:00<?, ?it/s]

H:\Desktop\Environement_Trading_Developement\Indicateur\s_score.py:31: RuntimeWarning:

invalid value encountered in sqrt

c:\Users\CYTech Student\AppData\Local\Programs\Python\Python311\Lib\site-packages\arch\univariate\base.py:768: ConvergenceWarning:

The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.


Calcul de la volatilité conditionnelle: 100%|██████████| 13273/13273 [00:00<00:00, 336386.74it/s]


13272


C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_14408\2552076431.py:23: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [40]:
strat = Strategie(DATA,tickers,"240m","240m",0)
port = strat.backtest(s_score_value_and_market_regime)

C:\Users\CYTech Student\AppData\Local\Temp\ipykernel_14408\2991588433.py:134: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.




------------------------------------------------------------------DETAILS : GPDUSD-----------------------------------------------------------------

Total Return : 0.31%
Benchmark return : 0.74%
CAGR : 0.05%
Volatility : 0.07%
Sharpe Ratio : 0.74
Benchmark sharpe ratio : 0.03
Max Drawdown: -0.16%
Calmar Ratio : 0.31
Beta : -0.0


Total trades : 33
Total long trades 14
Total short trades 19
Win rate : 0.58%
Wining streak : 5.0
Loosing streak : 3.0
Average winning trade : 3.0%
Average losing trade : -2.0%
Profit factor : 1.94
Expectancy : 9.37%



 -> Average duration trades (jour) : 35.46




--------------------------------------------------------------------------------------------------------------------------------------------------

